Import all necessary libraries.

In [ ]:
from qiskit.circuit import QuantumCircuit, QuantumRegister, AncillaRegister, Parameter
from qiskit.quantum_info import Statevector, Operator
import matplotlib.pyplot as plt
import numpy as np

First consider the case where n=2, then we are dealing with a 4-4 sudoku puzzle here.
For the purpose of testing, let the puzzle be (generated by AI):
[1, 0, 0, 4]
[0, 3, 0, 0] 
[0, 0, 1, 0]
[2, 0, 0, 3],
where the cells that show 0 are the cells not filled-in.

In [ ]:
puzzle = np.array([
        [1, 0, 0, 4],
        [0, 3, 0, 0], 
        [0, 0, 1, 0],
        [2, 0, 0, 3]
    ])
print(puzzle)
pzl_flt = puzzle.flatten()
print(pzl_flt)

Initialize the puzzle in quantum circuit using classical approach.
The puzzle_qinit method takes the flattened puzzle as input and outputs a initialized quantum circuit.
Treating the whole puzzle as a quantum state, we have potentially 4^16 number of states, which is 2^32, and hence we need 32 qubits to represent every possible state.
For the encoding of the values, I choose the order with the tensor product, for the consideration that the built-in statevector function might be used somewhere and this is more consistent.
Encode:
value 1 is represented by |00>
value 2 is represented by |10>
value 3 is represented by |01>
value 4 is represented by |11>
value 0 means it is empty

In [72]:
def puzzle_qinit(pzlFlt):
    # construct a 32-qubit register all initialized to |0>
    qReg = QuantumRegister(32, "puzzle")
    ancReg = AncillaRegister(500, "ancilla")
    qPuzzle = QuantumCircuit(qReg, ancReg)
    # initialize circuit index as 0, since every cell has 4 states, we process two qubits at a time.
    cirIdx = 0
    for value in pzlFlt:
        if value == 1:
            # nothing needs to be done since |00> already represents 1
            pass
        elif value == 2:
            # flip the first bit to get |10>
            qPuzzle.x(qReg[cirIdx])
        elif value == 3:
            # flip the second bit to get |01>
            qPuzzle.x(qReg[cirIdx+1])
        elif value == 4:
            # flip both bits to get |11>
            qPuzzle.x(qReg[cirIdx])
            qPuzzle.x(qReg[cirIdx+1])
        elif value == 0:
            # apply Hadamard gate to empty cell bits
            qPuzzle.h(qReg[cirIdx])
            qPuzzle.h(qReg[cirIdx+1])
        # no else for error handling for simplicity at the moment
        cirIdx += 2
    return qPuzzle

q_puzzle = puzzle_qinit(pzl_flt)
#q_puzzle.draw(output="mpl", style="bw")

A helper function for the Oracle: sudoku_iter, takes in an integer and spits out a 4-number array indicating which 4 values in the flattened puzzle to check the sudoku constraint. The order is: row0, row1, row2, row3, col0, col1, col2, col3, square00, square01, square 10, square11.
For instance:
sudoku_iter(0) would return [0,1,2,3]
sudoku_iter(7) would return [3,7,11,15]
sudoku_iter(8) would return [0,1,4,5] 

In [ ]:
def sudoku_iter(idx):
    helperArr = np.array(range(16)).reshape(4,4)
    if idx < 4:
        return helperArr[idx, :].flatten()
    elif idx < 8:
        return helperArr[:, idx-4].flatten()
    else:
        # the second index on the square is always two times the remainder of the index divided by 2.
        # use that to determine starting points of slicing on both.
        t2 = idx%2
        t1 = idx-8-t2
        t2 = t2*2
        return helperArr[t1:t1+2, t2:t2+2].flatten()

# tests    
#print(sudoku_iter(0))
#print(sudoku_iter(5))
#print(sudoku_iter(8))
print(sudoku_iter(11))

Next up to construct the Oracle:
For the "f" in the Oracle, I plan to map the solved sudoku as 1 and otherwise 0
1. determine which 4 qubits we perform the constraint check on
2. performce XOR operation between bits

In [ ]:
def oracle_f(qPuzzle):
    qReg = qPuzzle.qregs[0]
    ancReg = qPuzzle.qregs[1]
    ancIdx = 0
    andRes = []
    consRes = []
    pairs = [[0,1],[0,2],[0,3],[1,2],[1,3],[2,3]]
    # we have overall 12 constrains in a 4-4 sudoku puzzle
    for i in range(12):
        pickedCells = sudoku_iter(i)
        pickedQbits = []
        for j in range(4):
            tmp = pickedCells[j]*2
            pickedQbits.append([qReg[tmp],qReg[tmp+1]])
        # perform cx on ancilla bits with puzzle bit as control to perform XOR and show if there is a difference
        # let pickedQbits be [q0,q1,q2,q3], then this is comparison between q0 and q1
        for pair in pairs:
            i1 = pair[0]
            i2 = pair[1]
            qPuzzle.cx(pickedQbits[i1][0], ancReg[ancIdx])
            qPuzzle.cx(pickedQbits[i2][0], ancReg[ancIdx])
            qPuzzle.cx(pickedQbits[i1][1], ancReg[ancIdx+1])
            qPuzzle.cx(pickedQbits[i2][1], ancReg[ancIdx+1])
            # q0!=q1 if either qubit disagrees, to perform an OR gate here, use De Morgan's law, a1|a2=!(!a1&!a2)
            # to perform an AND, take advantage of the CCX gate, the target bit flips iff both control bits are 1,
            # which has an underlying AND operation.
            qPuzzle.x(ancReg[ancIdx])
            qPuzzle.x(ancReg[ancIdx+1])
            qPuzzle.ccx(ancReg[ancIdx], ancReg[ancIdx+1], ancReg[ancIdx+2])
            qPuzzle.x(ancReg[ancIdx+2])
            andRes.append(ancReg[ancIdx+2])
            # restore used ancilla bits to original state for safety
            qPuzzle.x(ancReg[ancIdx])
            qPuzzle.x(ancReg[ancIdx+1])
            ancIdx += 3

        # perform AND between the 6 comparisons to conclude that the constraint has been met
        # again, use CCX to perform AND, but use output of first pair as input of second pair
        andIN1 = andRes[0]
        for j in range(5):
            andIN2 = andRes[j+1]
            qPuzzle.ccx(andIN1, andIN2, ancReg[ancIdx])
            andIN1 = ancReg[ancIdx]
            ancIdx += 1
        consRes.append(andIN1)
    
    # with results of all 12 constraints, we can perform a 12-way AND to determine if a sudoku solution has been found
    andIN1 = consRes[0]
    for i in range(11):
        andIN2 = consRes[i+1]
        qPuzzle.ccx(andIN1, andIN2, ancReg[ancIdx])
        andIN2 = ancReg[ancIdx]
        ancIdx += 1
    oracleRes = andIN1

    return 

oracle_f(q_puzzle)

2